In [1]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import pandas as pd
import json
import wandb
import os

/home/kyobrien/conda/envs/eval-aug/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
project_name = "healthyml/In-Context Domain Transfer Improves Out-of-Domain Robustness"
table_paths = {
    # BOSS Sentiment
    # ID
    # BERT
    "BOSS_Sentiment_ID_BERT_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Substitute": "",
    "BOSS_Sentiment_ID_BERT_Translate": "",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    # 1500, 3000, 6000, 12000, 24000
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_Insert": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_Translate": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_ICR": "",

    "BOSS_Sentiment_Ablate_Data_ID_BERT3000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT3000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT3000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT3000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT3000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_ID_BERT6000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT6000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT6000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT6000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT6000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_ID_BERT120000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT120000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT120000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT120000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT120000_ICR": "",

    "BOSS_Sentiment_Ablate_Data_ID_BERT24000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT24000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT24000_Translate": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT24000_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT24000_ICR": "",

    # T5
    "BOSS_Sentiment_ID_T5_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Substitute": "run-vj8tss18-validation_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Translate": "run-vj8tss18-validation_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_Paraphrase": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_ID_T5_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_ID_Falcon_Insert": "run-j11olfw3-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Substitute": "run-j11olfw3-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Translate": "run-j11olfw3-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_ID_Falcon_Paraphrase": "",
    "BOSS_Sentiment_ID_Falcon_ICR": "",

    # SST5
    # BERT
    "BOSS_Sentiment_SST5_BERT_Insert": "run-ibb61kcx-sst5_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Substitute": "run-ibb61kcx-sst5_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Translate": "run-ibb61kcx-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_Paraphrase": "run-ibb61kcx-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SST5_BERT_ICR": "run-ibb61kcx-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert": "run-gdqehris-sst5_aug_insert_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute": "run-gdqehris-sst5_aug_substitute_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Translate": "run-gdqehris-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Paraphrase": "run-gdqehris-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT1500_ICR": "run-gdqehris-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Insert": "run-gfb8nevp-sst5_aug_insert_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Substitute": "run-gfb8nevp-sst5_aug_substitute_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Translate": "run-gfb8nevp-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Paraphrase": "run-gfb8nevp-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT3000_ICR": "run-gfb8nevp-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Insert": "run-hg6u172t-sst5_aug_insert_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Substitute": "run-hg6u172t-sst5_aug_substitute_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Translate": "run-hg6u172t-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_Paraphrase": "run-hg6u172t-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT6000_ICR": "run-hg6u172t-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT120000_Insert": "run-pv6dxpzt-sst5_aug_insert_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT120000_Substitute": "run-pv6dxpzt-sst5_aug_substitute_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT120000_Translate": "run-pv6dxpzt-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT120000_Paraphrase": "run-pv6dxpzt-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT120000_ICR": "run-pv6dxpzt-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Insert": "run-ujjjnolk-sst5_aug_insert_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Substitute": "run-ujjjnolk-sst5_aug_substitute_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Translate": "run-ujjjnolk-sst5_aug_backtranslate_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_Paraphrase": "run-ujjjnolk-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SST5_BERT24000_ICR": "run-ujjjnolk-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_SST5_T5_Insert": "run-hhtbcw8x-sst5_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Substitute": "run-hhtbcw8x-sst5_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Translate": "run-hhtbcw8x-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_Paraphrase": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SST5_T5_ICR": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_SST5_Falcon_Insert": "run-hhtbcw8x-sst5_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Substitute": "run-hhtbcw8x-sst5_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Translate": "run-hhtbcw8x-sst5_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_Paraphrase": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SST5_Falcon_ICR": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # SemEval
    # BERT
    "BOSS_Sentiment_SemEval_BERT_Insert": "run-ibb61kcx-semval_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Substitute": "run-ibb61kcx-semval_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Translate": "run-ibb61kcx-semval_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_Paraphrase": "run-ibb61kcx-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_SemEval_BERT_ICR": "run-ibb61kcx-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Insert": "run-gdqehris-semval_aug_insert_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Substitute": "run-gdqehris-semval_aug_substitute_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Translate": "run-gdqehris-semval_aug_backtranslate_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_Paraphrase": "run-gdqehris-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT1500_ICR": "run-gdqehris-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Insert": "run-gfb8nevp-semval_aug_insert_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Substitute": "run-gfb8nevp-semval_aug_substitute_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Translate": "run-gfb8nevp-semval_aug_backtranslate_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_Paraphrase": "run-gfb8nevp-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT3000_ICR": "run-gfb8nevp-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Insert": "run-hg6u172t-semval_aug_insert_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Substitute": "run-hg6u172t-semval_aug_substitute_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Translate": "run-hg6u172t-semval_aug_backtranslate_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_Paraphrase": "run-hg6u172t-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT6000_ICR": "run-hg6u172t-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT120000_Insert": "run-pv6dxpzt-semval_aug_insert_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT120000_Substitute": "run-pv6dxpzt-semval_aug_substitute_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT120000_Translate": "run-pv6dxpzt-semval_aug_backtranslate_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT120000_Paraphrase": "run-pv6dxpzt-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT120000_ICR": "run-pv6dxpzt-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Insert": "run-ujjjnolk-semval_aug_insert_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Substitute": "run-ujjjnolk-semval_aug_substitute_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Translate": "run-ujjjnolk-semval_aug_backtranslate_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_Paraphrase": "run-ujjjnolk-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_SemEval_BERT24000_ICR": "run-ujjjnolk-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_SemEval_T5_Insert": "run-hhtbcw8x-semval_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Substitute": "run-hhtbcw8x-semval_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Translate": "run-hhtbcw8x-semval_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_Paraphrase": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_SemEval_T5_ICR": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_SemEval_Falcon_Insert": "run-hhtbcw8x-semval_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Substitute": "run-hhtbcw8x-semval_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Translate": "run-hhtbcw8x-semval_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_Paraphrase": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_SemEval_Falcon_ICR": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # Dynasent
    # BERT
    "BOSS_Sentiment_Dynasent_BERT_Insert": "run-ibb61kcx-dynasent_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Substitute": "run-ibb61kcx-dynasent_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Translate": "run-ibb61kcx-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_Paraphrase": "run-ibb61kcx-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Dynasent_BERT_ICR": "run-ibb61kcx-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Insert": "run-gdqehris-dynasent_aug_insert_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Substitute": "run-gdqehris-dynasent_aug_substitute_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Translate": "run-gdqehris-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_Paraphrase": "run-gdqehris-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT1500_ICR": "run-gdqehris-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment1500bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Insert": "run-gfb8nevp-dynasent_aug_insert_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Substitute": "run-gfb8nevp-dynasent_aug_substitute_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Translate": "run-gfb8nevp-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_Paraphrase": "run-gfb8nevp-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT3000_ICR": "run-gfb8nevp-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment3000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Insert": "run-hg6u172t-dynasent_aug_insert_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Substitute": "run-hg6u172t-dynasent_aug_substitute_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Translate": "run-hg6u172t-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_Paraphrase": "run-hg6u172t-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT6000_ICR": "run-hg6u172t-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment6000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT120000_Insert": "run-pv6dxpzt-dynasent_aug_insert_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT120000_Substitute": "run-pv6dxpzt-dynasent_aug_substitute_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT120000_Translate": "run-pv6dxpzt-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT120000_Paraphrase": "run-pv6dxpzt-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT120000_ICR": "run-pv6dxpzt-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment12000bertbaseuncased_style_logs:v0",

    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Insert": "run-ujjjnolk-dynasent_aug_insert_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Substitute": "run-ujjjnolk-dynasent_aug_substitute_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Translate": "run-ujjjnolk-dynasent_aug_backtranslate_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_Paraphrase": "run-ujjjnolk-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_Dynasent_BERT24000_ICR": "run-ujjjnolk-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentiment24000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Sentiment_Dynasent_T5_Insert": "run-hhtbcw8x-dynasent_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Substitute": "run-hhtbcw8x-dynasent_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Translate": "run-hhtbcw8x-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_Paraphrase": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
    "BOSS_Sentiment_Dynasent_T5_ICR": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

    # Falcon
    "BOSS_Sentiment_Dynasent_Falcon_Insert": "run-hhtbcw8x-dynasent_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Substitute": "run-hhtbcw8x-dynasent_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Translate": "run-hhtbcw8x-dynasent_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_Paraphrase": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Sentiment_Dynasent_Falcon_ICR": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # BOSS Toxicity
    # ID
    # BERT
    "BOSS_Toxicity_ID_BERT_Insert": "",
    "BOSS_Toxicity_ID_BERT_Substitute": "",
    "BOSS_Toxicity_ID_BERT_Translate": "",
    "BOSS_Toxicity_ID_BERT_Paraphrase": "",
    "BOSS_Toxicity_ID_BERT_ICR": "run-dvdcuuij-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ID_BERT3000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT3000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT3000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT3000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT3000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ID_BERT6000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT6000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT6000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT6000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT6000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ID_BERT12000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT12000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT12000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT12000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT12000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ID_BERT24000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT24000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT24000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT24000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT24000_ICR": "",

    "BOSS_Toxicity_Ablate_Data_ID_BERT48000_Insert": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT48000_Substitute": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT48000_Translate": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT48000_Paraphrase": "",
    "BOSS_Toxicity_Ablate_Data_ID_BERT48000_ICR": "",

    # T5
    "BOSS_Toxicity_ID_T5_Insert": "",
    "BOSS_Toxicity_ID_T5_Substitute": "",
    "BOSS_Toxicity_ID_T5_Translate": "",
    "BOSS_Toxicity_ID_T5_Paraphrase": "",
    "BOSS_Toxicity_ID_T5_ICR": "",

    # Falcon
    "BOSS_Toxicity_ID_Falcon_Insert": "run-bo8gbtun-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ID_Falcon_Substitute": "",
    "BOSS_Toxicity_ID_Falcon_Translate": "",
    "BOSS_Toxicity_ID_Falcon_Paraphrase": "",
    "BOSS_Toxicity_ID_Falcon_ICR": "",

    # Toxigen
    # BERT
    "BOSS_Toxicity_Toxigen_BERT_Insert": "run-56jek2cf-toxigen_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Substitute": "run-56jek2cf-toxigen_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Translate": "run-56jek2cf-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_Paraphrase": "run-56jek2cf-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Toxigen_BERT_ICR": "run-56jek2cf-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Insert": "run-2ecsgmy9-toxigen_aug_insert_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Substitute": "run-2ecsgmy9-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Translate": "run-2ecsgmy9-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_Paraphrase": "run-2ecsgmy9-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT3000_ICR": "run-2ecsgmy9-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Insert": "run-ms2943fk-toxigen_aug_insert_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Substitute": "run-ms2943fk-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Translate": "run-ms2943fk-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_Paraphrase": "run-ms2943fk-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT6000_ICR": "run-ms2943fk-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Insert": "run-7hgtyr98-toxigen_aug_insert_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Substitute": "run-7hgtyr98-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Translate": "run-7hgtyr98-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_Paraphrase": "run-7hgtyr98-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT12000_ICR": "run-7hgtyr98-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Insert": "run-wl2a38kf-toxigen_aug_insert_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Substitute": "run-wl2a38kf-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Translate": "run-wl2a38kf-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_Paraphrase": "run-wl2a38kf-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT24000_ICR": "run-wl2a38kf-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Insert": "run-iaorx9o7-toxigen_aug_insert_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Substitute": "run-iaorx9o7-toxigen_aug_substitute_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Translate": "run-iaorx9o7-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_Paraphrase": "run-iaorx9o7-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_Toxigen_BERT48000_ICR": "run-iaorx9o7-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_Toxigen_T5_Insert": "run-lixlm6qh-toxigen_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Substitute": "run-lixlm6qh-toxigen_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Translate": "run-lixlm6qh-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_Paraphrase": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_Toxigen_T5_ICR": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_Toxigen_Falcon_Insert": "run-lixlm6qh-toxigen_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Substitute": "run-lixlm6qh-toxigen_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Translate": "run-lixlm6qh-toxigen_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_Paraphrase": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_Toxigen_Falcon_ICR": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # ImplicitHate
    # BERT
    "BOSS_Toxicity_ImplicitHate_BERT_Insert": "run-56jek2cf-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Substitute": "run-56jek2cf-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Translate": "run-56jek2cf-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_Paraphrase": "run-56jek2cf-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_BERT_ICR": "run-56jek2cf-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Insert": "run-ms2943fk-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Substitute": "run-ms2943fk-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Translate": "run-ms2943fk-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_Paraphrase": "run-ms2943fk-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT3000_ICR": "run-ms2943fk-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Insert": "run-2ecsgmy9-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Substitute": "run-2ecsgmy9-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Translate": "run-2ecsgmy9-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_Paraphrase": "run-2ecsgmy9-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT6000_ICR": "run-2ecsgmy9-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Insert": "run-7hgtyr98-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Substitute": "run-7hgtyr98-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Translate": "run-7hgtyr98-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_Paraphrase": "run-7hgtyr98-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT12000_ICR": "run-7hgtyr98-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Insert": "run-wl2a38kf-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Substitute": "run-wl2a38kf-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Translate": "run-wl2a38kf-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_Paraphrase": "run-wl2a38kf-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT24000_ICR": "run-wl2a38kf-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Insert": "run-iaorx9o7-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Substitute": "run-iaorx9o7-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Translate": "run-iaorx9o7-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_Paraphrase": "run-iaorx9o7-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_ImplicitHate_BERT48000_ICR": "run-iaorx9o7-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_ImplicitHate_T5_Insert": "run-lixlm6qh-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Substitute": "run-lixlm6qh-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Translate": "run-lixlm6qh-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_Paraphrase": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_T5_ICR": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_ImplicitHate_Falcon_Insert": "run-lixlm6qh-implicit_hate_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Substitute": "run-lixlm6qh-implicit_hate_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Translate": "run-lixlm6qh-implicit_hate_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_ImplicitHate_Falcon_Paraphrase": "",
    "BOSS_Toxicity_ImplicitHate_Falcon_ICR": "",

    # AdvCivil
    # BERT
    "BOSS_Toxicity_AdvCivil_BERT_Insert": "run-56jek2cf-adv_civil_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Substitute": "run-56jek2cf-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Translate": "run-56jek2cf-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_Paraphrase": "run-56jek2cf-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_BERT_ICR": "run-56jek2cf-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Insert": "run-ms2943fk-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Substitute": "run-ms2943fk-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Translate": "run-ms2943fk-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_Paraphrase": "run-ms2943fk-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT3000_ICR": "run-ms2943fk-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity3000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Insert": "run-2ecsgmy9-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Substitute": "run-2ecsgmy9-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Translate": "run-2ecsgmy9-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_Paraphrase": "run-2ecsgmy9-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT6000_ICR": "run-2ecsgmy9-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity6000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Insert": "run-7hgtyr98-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Substitute": "run-7hgtyr98-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Translate": "run-7hgtyr98-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_Paraphrase": "run-7hgtyr98-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT12000_ICR": "run-7hgtyr98-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity12000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Insert": "run-wl2a38kf-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Substitute": "run-wl2a38kf-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Translate": "run-wl2a38kf-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_Paraphrase": "run-wl2a38kf-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT24000_ICR": "run-wl2a38kf-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity24000bertbaseuncased_style_logs:v0",

    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Insert": "run-iaorx9o7-adv_civil_aug_insert_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Substitute": "run-iaorx9o7-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Translate": "run-iaorx9o7-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_Paraphrase": "run-iaorx9o7-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",
    "BOSS_Toxicity_Ablate_Data_AdvCivil_BERT48000_ICR": "run-iaorx9o7-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicity48000bertbaseuncased_style_logs:v0",

    # T5
    "BOSS_Toxicity_AdvCivil_T5_Insert": "run-lixlm6qh-adv_civil_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Substitute": "run-lixlm6qh-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Translate": "run-lixlm6qh-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_Paraphrase": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_T5_ICR": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

    # Falcon
    "BOSS_Toxicity_AdvCivil_Falcon_Insert": "run-lixlm6qh-adv_civil_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Substitute": "run-lixlm6qh-adv_civil_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Translate": "run-lixlm6qh-adv_civil_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_Paraphrase": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "BOSS_Toxicity_AdvCivil_Falcon_ICR": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # AG News Tweets
    # ID
    # BERT
    "AgNewsTweets_ID_BERT_Insert": "run-i2vkae2s-validation_aug_insert_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_ID_BERT_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_ICR": "",

    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Insert": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Substitute": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Translate": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Paraphrase": "",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_ICR": "",

    # T5
    "AgNewsTweets_ID_T5_Insert": "run-i2vkae2s-validation_aug_insert_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_ID_T5_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_Kyle1668agnewst5large_style_logs:v0",

    # Falcon
    "AgNewsTweets_ID_Falcon_Insert": "run-i2vkae2s-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_ID_Falcon_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

    # Tweets
    # BERT
    "AgNewsTweets_Tweets_BERT_Insert": "run-itucov43-test_aug_insert_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Substitute": "run-itucov43-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Translate": "run-itucov43-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_Paraphrase": "run-itucov43-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
    "AgNewsTweets_Tweets_BERT_ICR": "run-itucov43-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Insert": "run-seqenj7d-test_aug_insert_random_16_Kyle1668agnews76800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Substitute": "run-seqenj7d-test_aug_substitute_random_16_Kyle1668agnews76800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Translate": "run-seqenj7d-test_aug_backtranslate_random_16_Kyle1668agnews76800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_Paraphrase": "run-seqenj7d-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews76800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT76800_ICR": "run-seqenj7d-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnews76800bertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Insert": "run-ymlu6tfz-test_aug_insert_random_16_Kyle1668agnews9600bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Substitute": "run-ymlu6tfz-test_aug_substitute_random_16_Kyle1668agnews9600bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Translate": "run-ymlu6tfz-test_aug_backtranslate_random_16_Kyle1668agnews9600bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_Paraphrase": "run-ymlu6tfz-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews9600bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT9600_ICR": "run-ymlu6tfz-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews9600bertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Insert": "run-arskmac6-test_aug_insert_random_16_Kyle1668agnews19200bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Substitute": "run-arskmac6-test_aug_substitute_random_16_Kyle1668agnews19200bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Translate": "run-arskmac6-test_aug_backtranslate_random_16_Kyle1668agnews19200bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_Paraphrase": "run-arskmac6-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews19200bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT19200_ICR": "run-arskmac6-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnews19200bertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Insert": "run-6qktrbp8-test_aug_insert_random_16_Kyle1668agnews38400bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Substitute": "run-6qktrbp8-test_aug_substitute_random_16_Kyle1668agnews38400bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Translate": "run-6qktrbp8-test_aug_backtranslate_random_16_Kyle1668agnews38400bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_Paraphrase": "run-6qktrbp8-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews38400bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT38400_ICR": "run-6qktrbp8-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnews38400bertbaseuncased_style_logs:v0",

    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Insert": "run-s51a9946-test_aug_insert_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Substitute": "run-s51a9946-test_aug_substitute_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Translate": "run-s51a9946-test_aug_backtranslate_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_Paraphrase": "run-s51a9946-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnews4800bertbaseuncased_style_logs:v0",
    "AgNewsTweets_Ablate_Data_Tweets_BERT4800_ICR": "run-s51a9946-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnews4800bertbaseuncased_style_logs:v0",

    # T5
    "AgNewsTweets_Tweets_T5_Insert": "run-xdipkawo-test_aug_insert_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Substitute": "run-xdipkawo-test_aug_substitute_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Translate": "run-xdipkawo-test_aug_backtranslate_random_16_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_Paraphrase": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewst5large_style_logs:v0",
    "AgNewsTweets_Tweets_T5_ICR": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewst5large_style_logs:v0",

    # Falcon
    "AgNewsTweets_Tweets_Falcon_Insert": "run-xdipkawo-test_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Substitute": "run-xdipkawo-test_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Translate": "run-xdipkawo-test_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_Paraphrase": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
    "AgNewsTweets_Tweets_Falcon_ICR": "",
}

print(json.dumps(table_paths, indent=4))

{
    "BOSS_Sentiment_ID_BERT_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_Substitute": "",
    "BOSS_Sentiment_ID_BERT_Translate": "",
    "BOSS_Sentiment_ID_BERT_Paraphrase": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_Insert": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_Translate": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_Paraphrase": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT1500_ICR": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT3000_Insert": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT3000_Substitute": "",
    "BOSS_Sentiment_Ablate_Data_ID_BERT3000_Translate": "",


In [3]:
run = wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: kyledevinobrien1 (healthyml). Use `wandb login --relogin` to force relogin


In [4]:
def get_local_logs_path(directory):
    try:
        for contents in os.listdir(directory):
            if contents.endswith("style_logs.table.json"):
                return os.path.join(directory, contents)

            if os.path.isdir(os.path.join(directory, contents)):
                return get_local_logs_path(os.path.join(directory, contents))
    except:
        # print("No style logs found in {}".format(directory))
        return None

print(get_local_logs_path("data/BOSS_Sentiment_Dynasent_BERT_Insert"))

data/BOSS_Sentiment_Dynasent_BERT_Insert/dynasent_aug_insert_random_16_Kyle1668/boss-sentiment-bert-base-uncased_style_logs.table.json


In [5]:
# for each table path, run.use_artifact and download the artifact.
local_paths = {}
missing_tables = []
for table_name, table_path in tqdm(list(table_paths.items())[:]):
    try:
        if table_name == "BOSS_Sentiment_SST5_BERT_ICR":
            print()

        if table_path == "":
            missing_tables.append(table_name)
            continue

        if get_local_logs_path(f"data/{table_name}") is None:
            download_path = f"{project_name}/{table_path}"
            print(f"Downloading {table_name} --- {download_path}")
            artifact = run.use_artifact(download_path, type="run_table")
            artifact_dir = artifact.download(root=f"data/{table_name}")

        local_paths[table_name] = get_local_logs_path(f"data/{table_name}")
    except Exception as e:
        print(e)
        print(table_name)
        raise e
        # missing_tables.append(table_name)

print(f"\nMissing tables: {missing_tables}")
print(json.dumps(local_paths, indent=4))

100%|██████████| 375/375 [00:00<00:00, 22812.32it/s]



Missing tables: ['BOSS_Sentiment_ID_BERT_Substitute', 'BOSS_Sentiment_ID_BERT_Translate', 'BOSS_Sentiment_Ablate_Data_ID_BERT1500_Insert', 'BOSS_Sentiment_Ablate_Data_ID_BERT1500_Substitute', 'BOSS_Sentiment_Ablate_Data_ID_BERT1500_Translate', 'BOSS_Sentiment_Ablate_Data_ID_BERT1500_Paraphrase', 'BOSS_Sentiment_Ablate_Data_ID_BERT1500_ICR', 'BOSS_Sentiment_Ablate_Data_ID_BERT3000_Insert', 'BOSS_Sentiment_Ablate_Data_ID_BERT3000_Substitute', 'BOSS_Sentiment_Ablate_Data_ID_BERT3000_Translate', 'BOSS_Sentiment_Ablate_Data_ID_BERT3000_Paraphrase', 'BOSS_Sentiment_Ablate_Data_ID_BERT3000_ICR', 'BOSS_Sentiment_Ablate_Data_ID_BERT6000_Insert', 'BOSS_Sentiment_Ablate_Data_ID_BERT6000_Substitute', 'BOSS_Sentiment_Ablate_Data_ID_BERT6000_Translate', 'BOSS_Sentiment_Ablate_Data_ID_BERT6000_Paraphrase', 'BOSS_Sentiment_Ablate_Data_ID_BERT6000_ICR', 'BOSS_Sentiment_Ablate_Data_ID_BERT120000_Insert', 'BOSS_Sentiment_Ablate_Data_ID_BERT120000_Substitute', 'BOSS_Sentiment_Ablate_Data_ID_BERT120000_T

In [6]:
column_renaming = {
    "entropy": "tta_prediction_entropy",
    "mean probs": "tta_mean_class_probs",
    "all probs": "tta_all_class_probs",
    "all entropies": "tta_all_class_entropy",
    "latency": "tta_inference_latency",
    "input": "augmentations",
    "original_input": "original_text",
    "judgment": "tta_predicted_class",
    "original judgment": "original_predicted_class",
    "original entropy": "original_prediction_entropy",
    "entropy decrease": "prediction_entropy_decrease",
    "entropy decreased": "prediction_entropy_decreased",
}
columns_to_drop = ["style prompt", "mean exemplar distance", "prompt"]
columns_order = ["outcome", "original_text", "augmentations", "generations", "original_predicted_class", "tta_predicted_class", "label", "tta_inference_latency", "original_prediction_entropy", "tta_prediction_entropy", "prediction_entropy_decreased", "prediction_entropy_decrease", "tta_mean_class_probs", "tta_all_class_probs", "tta_all_class_entropy", ]

In [7]:
# list(missing_tables)

In [8]:
def parse_augmentations(augmentations):
    if augmentations is None or len(augmentations) == 0:
        return []

    return [aug.replace("<aug>", "").replace("</aug>", "") for aug in augmentations[:4]]

formatted_augs = parse_augmentations([ "<aug>too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.</aug>", "<aug>one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.</aug>", "<aug>too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.</aug>", "<aug>too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>", "<aug>Too much of this well-acted but dangerously slow thriller feels like a preamble to a bigger , more complicated story , one that never materializes .</aug>" ])

print(formatted_augs)
print(len(formatted_augs))

['too not much of anything this well - well acted but dangerously slow thriller feels somehow like simply a terrible preamble to a bigger, much more too complicated horror story, one reality that never materializes.', 'one too too much apprehension of this well - being acted but dangerously still slow thriller feels like only a preamble to a seemingly bigger, yet more more complicated story, and one that never materializes.', 'too much of this a well - well acted but dangerously slow thriller film feels like a big preamble opening to a somehow bigger, more complicated dream story, the one concept that almost never materializes.', 'too am much of this this well - acted but dangerously slow paced thriller feels somehow like a prime preamble to creating a bigger, more painfully complicated story, but one one that never ever materializes.']
4


In [10]:
combined_datset = DatasetDict()
for split in tqdm(local_paths):
    # print(f"Processing {split}")
    path = local_paths[split]
    json_logs = json.load(open(path))
    frame = pd.DataFrame(data=json_logs["data"], columns=json_logs["columns"])
    frame = frame.rename(columns=column_renaming)
    frame = frame.drop(columns=columns_to_drop)
    frame["augmentations"] = frame["augmentations"].apply(parse_augmentations)

    if "BERT" not in split:
        for col in columns_order:
            if col not in frame.columns:
                if col == "original_prediction_entropy" or col == "tta_prediction_entropy" or col == "prediction_entropy_decrease":
                    frame[col] = float("nan")
                elif col == "prediction_entropy_decreased":
                    frame[col] = False
                elif col == "tta_all_class_probs":
                    frame[col] = frame.apply(lambda x: [[float(0.0)]], axis=1)
                elif "probs" in col:
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)
                elif col == "tta_all_class_entropy":
                    frame[col] = frame.apply(lambda x: [float(0.0)], axis=1)

            if "generations" not in frame.columns:
                frame["generations"] = []


    frame = frame[columns_order]
    combined_datset[split] = Dataset.from_pandas(frame)

combined_datset

  0%|          | 0/305 [00:08<?, ?it/s]


KeyError: "['generations'] not in index"

In [20]:
combined_datset.save_to_disk("data/combined_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 7600/7600 [00:00<00:00, 346742.27 examples/s]


In [21]:
list(combined_datset.keys())

['BOSS_Sentiment_ID_BERT_Insert',
 'BOSS_Sentiment_ID_BERT_Paraphrase',
 'BOSS_Sentiment_ID_BERT_ICR',
 'BOSS_Sentiment_ID_T5_Insert',
 'BOSS_Sentiment_ID_T5_Substitute',
 'BOSS_Sentiment_ID_T5_Translate',
 'BOSS_Sentiment_ID_T5_Paraphrase',
 'BOSS_Sentiment_ID_T5_ICR',
 'BOSS_Sentiment_ID_Falcon_Insert',
 'BOSS_Sentiment_ID_Falcon_Substitute',
 'BOSS_Sentiment_ID_Falcon_Translate',
 'BOSS_Sentiment_SST5_BERT_Insert',
 'BOSS_Sentiment_SST5_BERT_Substitute',
 'BOSS_Sentiment_SST5_BERT_Translate',
 'BOSS_Sentiment_SST5_BERT_Paraphrase',
 'BOSS_Sentiment_SST5_BERT_ICR',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Insert',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Substitute',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Translate',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_Paraphrase',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT1500_ICR',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Insert',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Substitute',
 'BOSS_Sentiment_Ablate_Data_SST5_BERT3000_Transla

In [22]:
# combined_datset.push_to_hub("LLM-TTA-Augmentation-Logs")

## 11/11

In [ ]:
# project_name = "healthyml/In-Context Domain Transfer Improves Out-of-Domain Robustness"
# table_paths = {
#     # BOSS Sentiment
#     # ID
#     # BERT
#     "BOSS_Sentiment_ID_BERT_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_ID_BERT_Substitute": "",
#     "BOSS_Sentiment_ID_BERT_Translate": "",
#     "BOSS_Sentiment_ID_BERT_Paraphrase": "",
#     "BOSS_Sentiment_ID_BERT_ICR": "run-yanx1dus-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

#     # T5
#     "BOSS_Sentiment_ID_T5_Insert": "run-vj8tss18-validation_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_ID_T5_Substitute": "run-vj8tss18-validation_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_ID_T5_Translate": "run-vj8tss18-validation_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_ID_T5_Paraphrase": "",
#     "BOSS_Sentiment_ID_T5_ICR": "",

#     # Falcon
#     "BOSS_Sentiment_ID_Falcon_Insert": "run-j11olfw3-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_ID_Falcon_Substitute": "run-j11olfw3-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_ID_Falcon_Translate": "run-j11olfw3-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_ID_Falcon_Paraphrase": "",
#     "BOSS_Sentiment_ID_Falcon_ICR": "",

#     # SST5
#     # BERT
#     "BOSS_Sentiment_SST5_BERT_Insert": "run-hhtbcw8x-sst5_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_SST5_BERT_Substitute": "run-hhtbcw8x-sst5_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_SST5_BERT_Translate": "run-hhtbcw8x-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_SST5_BERT_Paraphrase": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_SST5_BERT_ICR": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

#     # T5
#     "BOSS_Sentiment_SST5_T5_Insert": "run-hhtbcw8x-sst5_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_SST5_T5_Substitute": "run-hhtbcw8x-sst5_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_SST5_T5_Translate": "run-hhtbcw8x-sst5_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_SST5_T5_Paraphrase": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_SST5_T5_ICR": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

#     # Falcon
#     "BOSS_Sentiment_SST5_Falcon_Insert": "run-hhtbcw8x-sst5_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_SST5_Falcon_Substitute": "run-hhtbcw8x-sst5_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_SST5_Falcon_Translate": "run-hhtbcw8x-sst5_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_SST5_Falcon_Paraphrase": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_SST5_Falcon_ICR": "run-hhtbcw8x-sst5_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

#     # SemEval
#     # BERT
#     "BOSS_Sentiment_SemEval_BERT_Insert": "run-hhtbcw8x-semval_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_SemEval_BERT_Substitute": "run-hhtbcw8x-semval_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_SemEval_BERT_Translate": "run-hhtbcw8x-semval_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_SemEval_BERT_Paraphrase": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_SemEval_BERT_ICR": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

#     # T5
#     "BOSS_Sentiment_SemEval_T5_Insert": "run-hhtbcw8x-semval_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_SemEval_T5_Substitute": "run-hhtbcw8x-semval_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_SemEval_T5_Translate": "run-hhtbcw8x-semval_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_SemEval_T5_Paraphrase": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_SemEval_T5_ICR": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

#     # Falcon
#     "BOSS_Sentiment_SemEval_Falcon_Insert": "run-hhtbcw8x-semval_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_SemEval_Falcon_Substitute": "run-hhtbcw8x-semval_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_SemEval_Falcon_Translate": "run-hhtbcw8x-semval_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_SemEval_Falcon_Paraphrase": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_SemEval_Falcon_ICR": "run-hhtbcw8x-semval_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

#     # Dynasent
#     # BERT
#     "BOSS_Sentiment_Dynasent_BERT_Insert": "run-hhtbcw8x-dynasent_aug_insert_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_BERT_Substitute": "run-hhtbcw8x-dynasent_aug_substitute_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_BERT_Translate": "run-hhtbcw8x-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_BERT_Paraphrase": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_BERT_ICR": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentbertbaseuncased_style_logs:v0",

#     # T5
#     "BOSS_Sentiment_Dynasent_T5_Insert": "run-hhtbcw8x-dynasent_aug_insert_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_T5_Substitute": "run-hhtbcw8x-dynasent_aug_substitute_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_T5_Translate": "run-hhtbcw8x-dynasent_aug_backtranslate_random_16_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_T5_Paraphrase": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_Kyle1668bosssentimentt5large_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_T5_ICR": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_Kyle1668bosssentimentt5large_style_logs:v0",

#     # Falcon
#     "BOSS_Sentiment_Dynasent_Falcon_Insert": "run-hhtbcw8x-dynasent_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_Falcon_Substitute": "run-hhtbcw8x-dynasent_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_Falcon_Translate": "run-hhtbcw8x-dynasent_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_Falcon_Paraphrase": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Sentiment_Dynasent_Falcon_ICR": "run-hhtbcw8x-dynasent_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

#     # BOSS Toxicity
#     # ID
#     # BERT
#     "BOSS_Toxicity_ID_BERT_Insert": "",
#     "BOSS_Toxicity_ID_BERT_Substitute": "",
#     "BOSS_Toxicity_ID_BERT_Translate": "",
#     "BOSS_Toxicity_ID_BERT_Paraphrase": "",
#     "BOSS_Toxicity_ID_BERT_ICR": "run-dvdcuuij-validation_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

#     # T5
#     "BOSS_Toxicity_ID_T5_Insert": "",
#     "BOSS_Toxicity_ID_T5_Substitute": "",
#     "BOSS_Toxicity_ID_T5_Translate": "",
#     "BOSS_Toxicity_ID_T5_Paraphrase": "",
#     "BOSS_Toxicity_ID_T5_ICR": "",

#     # Falcon
#     "BOSS_Toxicity_ID_Falcon_Insert": "run-bo8gbtun-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_ID_Falcon_Substitute": "",
#     "BOSS_Toxicity_ID_Falcon_Translate": "",
#     "BOSS_Toxicity_ID_Falcon_Paraphrase": "",
#     "BOSS_Toxicity_ID_Falcon_ICR": "",

#     # Toxigen
#     # BERT
#     "BOSS_Toxicity_Toxigen_BERT_Insert": "run-lixlm6qh-toxigen_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_BERT_Substitute": "run-lixlm6qh-toxigen_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_BERT_Translate": "run-lixlm6qh-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_BERT_Paraphrase": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_BERT_ICR": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

#     # T5
#     "BOSS_Toxicity_Toxigen_T5_Insert": "run-lixlm6qh-toxigen_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_T5_Substitute": "run-lixlm6qh-toxigen_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_T5_Translate": "run-lixlm6qh-toxigen_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_T5_Paraphrase": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_T5_ICR": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

#     # Falcon
#     "BOSS_Toxicity_Toxigen_Falcon_Insert": "run-lixlm6qh-toxigen_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_Falcon_Substitute": "run-lixlm6qh-toxigen_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_Falcon_Translate": "run-lixlm6qh-toxigen_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_Falcon_Paraphrase": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_Toxigen_Falcon_ICR": "run-lixlm6qh-toxigen_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

#     # ImplicitHate
#     # BERT
#     "BOSS_Toxicity_ImplicitHate_BERT_Insert": "run-lixlm6qh-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_BERT_Substitute": "run-lixlm6qh-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_BERT_Translate": "run-lixlm6qh-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_BERT_Paraphrase": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_BERT_ICR": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

#     # T5
#     "BOSS_Toxicity_ImplicitHate_T5_Insert": "run-lixlm6qh-implicit_hate_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_T5_Substitute": "run-lixlm6qh-implicit_hate_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_T5_Translate": "run-lixlm6qh-implicit_hate_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_T5_Paraphrase": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_T5_ICR": "run-lixlm6qh-implicit_hate_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

#     # Falcon
#     "BOSS_Toxicity_ImplicitHate_Falcon_Insert": "run-lixlm6qh-implicit_hate_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_Falcon_Substitute": "run-lixlm6qh-implicit_hate_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_Falcon_Translate": "run-lixlm6qh-implicit_hate_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_ImplicitHate_Falcon_Paraphrase": "",
#     "BOSS_Toxicity_ImplicitHate_Falcon_ICR": "",

#     # AdvCivil
#     # BERT
#     "BOSS_Toxicity_AdvCivil_BERT_Insert": "run-lixlm6qh-adv_civil_aug_insert_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_BERT_Substitute": "run-lixlm6qh-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_BERT_Translate": "run-lixlm6qh-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_BERT_Paraphrase": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_BERT_ICR": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicitybertbaseuncased_style_logs:v0",

#     # T5
#     "BOSS_Toxicity_AdvCivil_T5_Insert": "run-lixlm6qh-adv_civil_aug_insert_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_T5_Substitute": "run-lixlm6qh-adv_civil_aug_substitute_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_T5_Translate": "run-lixlm6qh-adv_civil_aug_backtranslate_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_T5_Paraphrase": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_0_Kyle1668bosstoxicityt5large_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_T5_ICR": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_16_Kyle1668bosstoxicityt5large_style_logs:v0",

#     # Falcon
#     "BOSS_Toxicity_AdvCivil_Falcon_Insert": "run-lixlm6qh-adv_civil_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_Falcon_Substitute": "run-lixlm6qh-adv_civil_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_Falcon_Translate": "run-lixlm6qh-adv_civil_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_Falcon_Paraphrase": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
#     "BOSS_Toxicity_AdvCivil_Falcon_ICR": "run-lixlm6qh-adv_civil_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

#     # AG News Tweets
#     # ID
#     # BERT
#     "AgNewsTweets_ID_BERT_Insert": "run-i2vkae2s-validation_aug_insert_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
#     "AgNewsTweets_ID_BERT_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
#     "AgNewsTweets_ID_BERT_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
#     "AgNewsTweets_ID_BERT_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
#     "AgNewsTweets_ID_BERT_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

#     # T5
#     "AgNewsTweets_ID_T5_Insert": "run-i2vkae2s-validation_aug_insert_random_16_Kyle1668agnewst5large_style_logs:v0",
#     "AgNewsTweets_ID_T5_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_Kyle1668agnewst5large_style_logs:v0",
#     "AgNewsTweets_ID_T5_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_Kyle1668agnewst5large_style_logs:v0",
#     "AgNewsTweets_ID_T5_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_Kyle1668agnewst5large_style_logs:v0",
#     "AgNewsTweets_ID_T5_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_Kyle1668agnewst5large_style_logs:v0",

#     # Falcon
#     "AgNewsTweets_ID_Falcon_Insert": "run-i2vkae2s-validation_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "AgNewsTweets_ID_Falcon_Substitute": "run-i2vkae2s-validation_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "AgNewsTweets_ID_Falcon_Translate": "run-i2vkae2s-validation_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "AgNewsTweets_ID_Falcon_Paraphrase": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
#     "AgNewsTweets_ID_Falcon_ICR": "run-i2vkae2s-validation_stabilityaiStableBeluga7b_random_16_tiiuaefalcon7binstruct_style_logs:v0",

#     # Tweets
#     # BERT
#     "AgNewsTweets_Tweets_BERT_Insert": "run-xdipkawo-test_aug_insert_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
#     "AgNewsTweets_Tweets_BERT_Substitute": "run-xdipkawo-test_aug_substitute_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
#     "AgNewsTweets_Tweets_BERT_Translate": "run-xdipkawo-test_aug_backtranslate_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",
#     "AgNewsTweets_Tweets_BERT_Paraphrase": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewsbertbaseuncased_style_logs:v0",
#     "AgNewsTweets_Tweets_BERT_ICR": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewsbertbaseuncased_style_logs:v0",

#     # T5
#     "AgNewsTweets_Tweets_T5_Insert": "run-xdipkawo-test_aug_insert_random_16_Kyle1668agnewst5large_style_logs:v0",
#     "AgNewsTweets_Tweets_T5_Substitute": "run-xdipkawo-test_aug_substitute_random_16_Kyle1668agnewst5large_style_logs:v0",
#     "AgNewsTweets_Tweets_T5_Translate": "run-xdipkawo-test_aug_backtranslate_random_16_Kyle1668agnewst5large_style_logs:v0",
#     "AgNewsTweets_Tweets_T5_Paraphrase": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_Kyle1668agnewst5large_style_logs:v0",
#     "AgNewsTweets_Tweets_T5_ICR": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_16_Kyle1668agnewst5large_style_logs:v0",

#     # Falcon
#     "AgNewsTweets_Tweets_Falcon_Insert": "run-xdipkawo-test_aug_insert_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "AgNewsTweets_Tweets_Falcon_Substitute": "run-xdipkawo-test_aug_substitute_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "AgNewsTweets_Tweets_Falcon_Translate": "run-xdipkawo-test_aug_backtranslate_random_16_tiiuaefalcon7binstruct_style_logs:v0",
#     "AgNewsTweets_Tweets_Falcon_Paraphrase": "run-xdipkawo-test_stabilityaiStableBeluga7b_random_0_tiiuaefalcon7binstruct_style_logs:v0",
#     "AgNewsTweets_Tweets_Falcon_ICR": "",
# }

# print(json.dumps(table_paths, indent=4))